---
layout: post
title: Student Teaching Tracker
comments: true
---

<style>
    #student-list {
        list-style: none;
        padding: 0;
        margin: 0;
        color: white !important;
    }

    .student-item {
        display: flex;
        flex-direction: column;
        margin-bottom: 20px;
        color: white !important;
    }

    .student-name {
        text-align: left;
        font-weight: bold;
        color: white !important;
    }

    .controls {
        display: flex;
        justify-content: space-between;
        align-items: center;
        color: white !important;
        margin-bottom: 5px;
    }



    .objectStyled {
        padding: 10px 15px;
        margin: 0;
        font-size: 16px;
        font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
        border: none;
        outline: none;
        color: white !important;
        background: #1e1e1e;
        border-radius: 10px;
    }

    .inputStyled {
        padding: 10px 15px;
        margin: 0;
        font-size: 16px;
        font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
        border: none;
        outline: none;
        color: white !important;
        background-color: #444444 !important;
        border-radius: 10px;
    }


    button {
        padding: 10px 15px;
        margin: 0 5px;
        font-size: 16px;
        cursor: pointer;
        font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
        color: white !important;
    }

    #control-panel {
        text-align: center;
        margin-top: 20px;
        color: white !important;
    }

    input::file-selector-button {
        font-weight: bold;
        color: dodgerblue;
        padding: 0.5em;
        border: thin solid grey;
        border-radius: 3px;
    }
</style>

<ul id="student-list"></ul>

<div id="control-panel">
    <button class="glow-on-hover-search" onclick="saveSession()">Save Scores</button>
    <button class="glow-on-hover-search" onclick="resetTracker()">Clear Scores</button>
    <button type="button" class="glow-on-hover-search"
        onclick="document.getElementById('file-input-students').click();">Upload Student Roster</button>
    <button type="button" class="glow-on-hover-search"
        onclick="document.getElementById('file-input-categories').click();">Upload Scoring Categories</button>
    <input type="file" id="file-input-students" onchange="loadFile(event, 'students')" hidden />
    <input type="file" id="file-input-categories" onchange="loadFile(event, 'categories')" hidden />
</div>

<script>
    let students = [];
    let categories = [];
    let gradingData = {};

    // load the files
    function loadFile(event, type) {
        const file = event.target.files[0];
        const reader = new FileReader();

        reader.onload = function (e) {
            const content = e.target.result;
            const data = content.split(/\r?\n/).filter(line => line.trim() !== ""); // parsing the data

            if (type === 'students') { // load student roster
                students = data;
                initializeTracker();
            } else if (type === 'categories') { // load categories
                categories = data;
                initializeTracker();
            }
        };
        reader.readAsText(file);
    }

    // create the html elements
    function initializeTracker() {
        // if no students or categories were uploaded...
        if (students.length === 0 || categories.length === 0) return;

        // create the student list
        const studentList = document.getElementById('student-list');
        studentList.innerHTML = '';

        // for each student...
        students.forEach(student => {
            // add the score category
            gradingData[student] = {};
            categories.forEach(category => {
                gradingData[student][category] = 0; // initialize all categories with 0
            });

            // create the html stuff for it and add the styles
            const li = document.createElement('li');
            li.className = 'student-item';
            li.innerHTML = `
                <span class="student-name">${student}</span>
                ${categories.map(category => `
                    <span class="controls">
                        <span>${category}:</span>
                        <input id="input-${student}-${category}" type="number" step="0.01" min="0" max="1" class="inputStyled" value="0">
                    </span>
                `).join('')}
            `;
            li.classList.add("objectStyled")
            studentList.appendChild(li);
        });
    }

    // calculate total score
    function calculateTotal(student) {
        let total = 0; // initial score total

        // go through each scoring category and tally up the points
        categories.forEach(category => {
            total += parseFloat(document.getElementById(`input-${student}-${category}`).value || 0);
        });

        return total.toFixed(2); // return the total points
    }

    // save the scores
    function saveSession() {
        let fileContent = students.map(student => { // loops through each student

            // for each student:
            const studentScores = categories.map(category => {
                // get the student's score for each category as a number
                // if no score provided, default to 0
                const score = parseFloat(document.getElementById(`input-${student}-${category}`).value || 0);

                // round all the scores to 2 decimal places, return them
                gradingData[student][category] = score;
                return `${category}: ${score.toFixed(2)}`;
            }).join('\n'); // add a new line

            // calculate total score for the student
            const totalScore = calculateTotal(student);
            return `${student} total score: ${totalScore}\n${studentScores}`;
        }).join('\n\n');

        // create a new text file called scoringReport.txt
        // save the data there
        const blob = new Blob([fileContent], { type: 'text/plain' });
        const link = document.createElement('a');
        link.href = URL.createObjectURL(blob);
        link.download = 'scoringReport.txt';
        link.click();
    }

    // reset the scores
    function resetTracker() {
        // go through each student and each category; set the points value to be 0
        students.forEach(student => {
            categories.forEach(category => {
                document.getElementById(`input-${student}-${category}`).value = 0;
            });
        });
    }
</script>

### Directions
1. Upload a .txt file with a list of student names seperated by newlines. See below for an example:
   1. Student 1
   2. Student 2
   3. Student 3
2. Upload a .txt file with a list of categories you want to grade, seperated by newlines. See below for an example:
   1. Participation
   3. Accuracy
   4. Work Shown
   5. Grade on Problem 1
   6. Grade on Problem 2
3. You can change the numbers based on the grade you want to give them.
4. When you're finished, click the Save Scores button to save this to a text file. In a real scenario, you could upload these grades into Synergy (or any other grading platform)
5. If you want to reset the number of points everyone has, click the Clear Scores button.